# セットアップ

In [64]:
!pip install requests
!pip install beautifulsoup4
!pip install janome
!pip install scikit-learn
!pip list

You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/2f/c3/431de428dee36c577dec2097130fc5da8ebf8b51b0349e85d1cfa2088595/scikit_learn-0.22.2-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/85/7a/ae480be23b768910a9327c33517ced4623ba88dc035f9ce0206657c353a9/scipy-1.4.1-cp37-cp37m-macosx_10_6_intel.whl
  Using cached https://files.pythonhosted.org/packages/2f/5b/2cc2b9285e8b2ca8d2c1e4a2cbf1b12d70a2488ea78170de1909bca725f2/numpy-1.18.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/pack

In [3]:
import sys,time
venv_packages = '/Users/kawahara.masahiro/org/devio/ml-tfidf/venv/lib/python3.7/site-packages'
if venv_packages not in sys.path:
    sys.path.append(venv_packages)

# 前処理: 情報取得

## 対象のURLリスト取得
- 対象のURLリストは "./contents/url_list.txt" に格納しています

In [4]:
url_list_path = "./contents/url_list.txt"
url_list = []
with open(url_list_path) as f:
    url_list = [l.replace('\n', '') for l in f.readlines()]
print("len(url_list): {}".format(len(url_list)))
print("url_list[0]: {}".format(url_list[0]))

len(url_list): 48
url_list[0]: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/


## マッピング作成
- インデックス
    - ブログURL
    - ブログテキスト
    - 他、分かち書きした結果など

In [49]:
mapping = {}
for i, url in enumerate(url_list):
    mapping[i] = {}
    mapping[i]["url"] = url
print("len(mapping): {}".format(len(mapping)))

len(mapping): 48


## ブログテキストのリストを取得
- 各URLリストから "get_blog_texts" 関数を使ってブログテキストを取得する

In [6]:
from util.get_blog_texts import get_blog_texts

In [50]:
blog_texts = []
for i in mapping.keys():
    url = mapping[i]["url"]
    print("i {} getting texts from: {}".format(i, url))
    mapping[i]["texts"] = get_blog_texts(url)
    time.sleep(1)

i 0 getting texts from: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
i 1 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-cisco-site-to-site-vpn/
i 2 getting texts from: https://dev.classmethod.jp/cloud/aws/higobashi-aws-11-deepracer/
i 3 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-297-input-output/
i 4 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-cdk-intro-nw/
i 5 getting texts from: https://dev.classmethod.jp/etc/summit-osaka-2019-299-basic/
i 6 getting texts from: https://dev.classmethod.jp/cloud/aws/vscode-botostubs-boto3/
i 7 getting texts from: https://dev.classmethod.jp/etc/site_vpn_azure_aws/
i 8 getting texts from: https://dev.classmethod.jp/cloud/aws/awsspec-by-jupyter/
i 9 getting texts from: https://dev.classmethod.jp/cloud/aws/deepracer-new-input-parameters/
i 10 getting texts from: https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-298-security/
i 11 getting texts from: http

In [54]:
print("len(mapping[0][texts]): {}".format(len(mapping[0]["texts"])))
print("len(mapping[0][texts][0]): {}".format(mapping[0]["texts"][0]))

len(mapping[0][texts]): 65
len(mapping[0][texts][0]): Application Load Balancer (ALB), Network Load Balancer (NLB) はターゲットにローカルIPアドレス を指定できます。VPCピアリング接続先のインスタンスや、Direct Connect・VPN接続先のオンプレのサーバーをターゲットグループに登録することができます。


# 前処理: テキスト
- 空白の削除
- 全て小文字化

In [69]:
for i in mapping.keys():
    mapping[i]["texts"] = [t.replace(' ','').lower() for t in mapping[i]["texts"]]

In [56]:
print("len(mapping[0][texts]): {}".format(len(mapping[0]["texts"])))
print("len(mapping[0][texts][0]): {}".format(mapping[0]["texts"][0]))

len(mapping[0][texts]): 65
len(mapping[0][texts][0]): APPLICATIONLOADBALANCER(ALB),NETWORKLOADBALANCER(NLB)はターゲットにローカルIPアドレスを指定できます。VPCピアリング接続先のインスタンスや、DIRECTCONNECT・VPN接続先のオンプレのサーバーをターゲットグループに登録することができます。


# janome で単語抽出
- 各テキストの単語を抽出
- mapping\[i\]\["nouns"\] に格納
## 参考
- [Janome documentation](https://mocobeta.github.io/janome/)
- [Janome API #POSStopFilter](https://mocobeta.github.io/janome/api/janome.html#janome.tokenfilter.POSStopFilter)
- [Janome API #Token](https://mocobeta.github.io/janome/api/janome.html#janome.tokenizer.Token)

In [22]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

In [29]:
tokenizer = Tokenizer()
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [57]:
test_tokens = [token for token in a.analyze(mapping[0]["texts"][0])]
print("len(test_tokens): {}".format(len(test_tokens)))
print("test_tokens[0]: {}".format(test_tokens[0]))
print("[t.surface for t in test_tokens[:5]]: {}".format(' '.join([t.surface for t in test_tokens[:5]])))

len(test_tokens): 28
test_tokens[0]: APPLICATIONLOADBALANCER	名詞,固有名詞,組織,*,*,*,APPLICATIONLOADBALANCER,*,*
[t.surface for t in test_tokens[:5]]: APPLICATIONLOADBALANCER ( ALB ), NETWORKLOADBALANCER


In [63]:
for i in mapping.keys():
    texts_flat = "。".join(mapping[i]["texts"])
    tokens = [t for t in a.analyze(texts_flat)]
    # 登録
    mapping[i]["nouns"] = ' '.join([t.surface for t in tokens])
print("mapping[0][nouns][:100]: {}".format(mapping[0]["nouns"][:100]))

mapping[0][nouns][:100]: APPLICATIONLOADBALANCER ( ALB ), NETWORKLOADBALANCER ( NLB ) ターゲット ローカル IP アドレス 指定 VPC ピアリング 接続 先 イン


# CountVectorizer で単語出現頻度を計算
- `features` に名詞一覧
- `mapping[i]["bow"]` に名詞出現数
## 参考
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
import random
vectorizer = CountVectorizer()

In [75]:
X = vectorizer.fit_transform([mapping[i]["nouns"] for i in mapping.keys()])

In [115]:
features = vectorizer.get_feature_names()
bows = X.toarray()
for i, bow in enumerate(bows):
    mapping[i]["bow"] = bow
print("5 samples of features: {}".format(random.sample(features, 5)))
print("bow of mapping[0]: {}".format(mapping[0]["bow"]))
print("url of mapping[0]: {}".format(mapping[0]["url"]))
print("list high(>=5) frequency words of mapping[0]")
for i, count in enumerate(mapping[0]["bow"]):
    if count >= 5:
        print("  - {}: {}({})".format(count, features[i], i))

5 samples of features: ['dstsrcstateconn', 'コンセプト', '09', '高い', '改善']
bow of mapping[0]: [1 0 1 ... 0 0 0]
url of mapping[0]: https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
list high(>=5) frequency words of mapping[0]
  - 13: alb(112)
  - 5: app(166)
  - 5: http(546)
  - 7: ip(603)
  - 5: vpc(1158)
  - 5: こと(1272)
  - 8: オンプレサーバー(1481)
  - 7: サーバー(1585)
  - 6: サービス(1586)
  - 11: ターゲット(1665)
  - 5: ローカル(1951)
  - 12: 作成(2046)
  - 6: 指定(2365)
  - 6: 接続(2369)
  - 6: 検証(2471)
  - 10: 構築(2481)
  - 9: 環境(2551)
  - 5: 通信(2759)


# TF値の計算式導出関数を作ってみる
## 参考
- [【技術解説】単語の重要度を測る？TF-IDFとOkapi BM25の計算方法とは](https://mieruca-ai.com/ai/tf-idf_okapi-bm25/)

In [172]:
def calc_tf(word, blog_index):
    """blog_index 番目のブログの word の TF値を算出する"""
    # word, blogがない場合は 0を返す
    if (word not in features) or (mapping.get(blog_index) == None):
        return 0.0
    # ある場合、計算
    # ### word の出現回数の和
    word_count = mapping[blog_index]["bow"][features.index(word)]
    if word_count == 0:
        return 0.0
    # ### 全ての単語の出現回数の和
    sum_of_words = sum(mapping[blog_index]["bow"])
    # TF値計計算
    return word_count/float(sum_of_words)

## 1blogの TF値を計算してみる

In [176]:
print("----- TF values of {}".format(mapping[0]["url"]))
mapping[0]["tfs"] = [calc_tf(word, 0) for word in features]
best10 = sorted(enumerate(mapping[0]["tfs"]), key=lambda x:x[1], reverse=True)[:10]
for i, tf in best10:
    print("{}\t{}".format(features[i], tf))

----- TF values of https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
alb	0.040498442367601244
作成	0.037383177570093455
ターゲット	0.03426791277258567
構築	0.03115264797507788
環境	0.028037383177570093
オンプレサーバー	0.024922118380062305
ip	0.021806853582554516
サーバー	0.021806853582554516
サービス	0.018691588785046728
指定	0.018691588785046728


## 全ブログの TF値を計算する

In [175]:
for i in mapping.keys():
    mapping[i]["tfs"] = [calc_tf(word, i) for word in features]

In [187]:
for i in mapping.keys():
    print("\n--- TOP 3 TF values of {}".format(mapping[i]["url"]))
    best3 = sorted(enumerate(mapping[i]["tfs"]), key=lambda x:x[1], reverse=True)[:5]
    for index, tf in best3:
        print("{}\t{}".format(features[index], tf))    


--- TOP 3 TF values of https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
alb	0.040498442367601244
作成	0.037383177570093455
ターゲット	0.03426791277258567
構築	0.03115264797507788
環境	0.028037383177570093

--- TOP 3 TF values of https://dev.classmethod.jp/cloud/aws/aws-cisco-site-to-site-vpn/
aws	0.041025641025641026
vpn	0.038461538461538464
自宅	0.035897435897435895
接続	0.03333333333333333
active	0.028205128205128206

--- TOP 3 TF values of https://dev.classmethod.jp/cloud/aws/higobashi-aws-11-deepracer/
aws	0.0449438202247191
料金	0.0449438202247191
awsdeepracer	0.033707865168539325
usd	0.033707865168539325
時間	0.033707865168539325

--- TOP 3 TF values of https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-297-input-output/
aws	0.06353591160220995
情報	0.04419889502762431
技術	0.03314917127071823
トレーニング	0.019337016574585635
最新	0.019337016574585635

--- TOP 3 TF values of https://dev.classmethod.jp/cloud/aws/aws-cdk-intro-nw/
作成	0.05813953488372093
リソース	0.03488372093023256


# IDFの計算式を作ってみる

In [179]:
import math

def calc_idf(word):    
    # 総文書数を計算
    N = len(mapping.keys())
    # word が登録されていない場合
    if word not in features:
        return math.log2(N)
    # word indexを取得
    word_index = features.index(word)
    # 単語 word が出現する文書数 df を計算
    df = len([i for i in mapping.keys() if mapping[i]["bow"][word_index] > 0])
    # idf を計算
    return math.log2(N/float(df + 1))

## 全単語のIDFを計算してみる

In [180]:
idfs = [calc_idf(word) for word in features]

In [181]:
best10  = sorted(enumerate(idfs), key=lambda x:x[1], reverse=True)[:10]
worst10 = sorted(enumerate(idfs), key=lambda x:x[1])[:10]
print("----- IDF values: BEST 10")
[print("{}\t{}".format(features[i],idf)) for i,idf in best10]
print("\n----- IDF values: WORST 10")
[print("{}\t{}".format(features[i],idf)) for i,idf in worst10]

----- IDF values: BEST 10
000	4.584962500721156
04	4.584962500721156
05	4.584962500721156
07	4.584962500721156
08	4.584962500721156
09	4.584962500721156
107	4.584962500721156
1234567890	4.584962500721156
149	4.584962500721156
160	4.584962500721156

----- IDF values: WORST 10
こと	0.2274104961030724
aws	0.41503749927884376
ため	0.49749965947081687
よう	0.49749965947081687
作成	0.49749965947081687
この	0.5405683813627028
以下	0.5405683813627028
今回	0.726981505593584
環境	0.726981505593584
利用	0.8300749985576875


[None, None, None, None, None, None, None, None, None, None]

# TF-IDF を計算してみる

In [184]:
for i in mapping.keys():
    mapping[i]["tfidfs"] = [idfs[index] * tf for index,tf in enumerate(mapping[i]["tfs"])]


In [186]:
for i in mapping.keys():
    print("\n--- TOP 3 TF-IDF values of {}".format(mapping[i]["url"]))
    best3 = sorted(enumerate(mapping[i]["tfidfs"]), key=lambda x:x[1], reverse=True)[:5]
    for index, tf in best3:
        print("{}\t{}".format(features[index], tf))    


--- TOP 3 TF-IDF values of https://dev.classmethod.jp/cloud/aws/on-premise-reverse-proxy-using-alb/
alb	0.12149532710280372
オンプレサーバー	0.11426697821111916
ターゲット	0.10280373831775699
サーバー	0.05636989876962022
http	0.05584053739441053

--- TOP 3 TF-IDF values of https://dev.classmethod.jp/cloud/aws/aws-cisco-site-to-site-vpn/
active	0.11282051282051282
自宅	0.10769230769230768
vpn	0.08175118777245612
cisco	0.07353769232248525
接続	0.06281742608600213

--- TOP 3 TF-IDF values of https://dev.classmethod.jp/cloud/aws/higobashi-aws-11-deepracer/
料金	0.16112191014477106
usd	0.15454929777711762
awsdeepracer	0.1348314606741573
cd	0.10303286518474508
ci	0.10303286518474508

--- TOP 3 TF-IDF values of https://dev.classmethod.jp/cloud/aws/aws-summit-osaka-2019-297-input-output/
技術	0.08005649168880145
トレーニング	0.07734806629834254
資料	0.07599385360311309
キャッチ	0.05066256906874205
最新	0.049985462721127326

--- TOP 3 TF-IDF values of https://dev.classmethod.jp/cloud/aws/aws-cdk-intro-nw/
nw	0.09329865553793051
pra